In [67]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

Using cache found in /home/mrtbuntu/.cache/torch/hub/pytorch_vision_v0.10.0


In [68]:
import os
root = "../../tp_final/notebooks/image_processing/img"
images_names =  os.listdir(root)
images_path = [f"{root}{image_name}" for image_name in images_names]

In [69]:
images_path

['/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/car_1.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/car_2.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/car_3.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/cat_1.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/cat_2.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/cat_3.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/truck.jpg',
 '/mnt/c/Users/mrspa/Documents/Repos/ITBA/TP Final Imagenes/tp_final/notebooks/image_processing/img/truck_original.jpg']

In [70]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
outputs = {}
for image_name, image_path in zip(images_names, images_path):
    input_image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
        print("using cuda")

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])
    outputs[image_name] = output[0]
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    # probabilities = torch.nn.functional.softmax(output[0], dim=0)
    # print(probabilities)

using cuda
using cuda
using cuda
using cuda
using cuda
using cuda
using cuda
using cuda


In [80]:
from scipy.spatial.distance import cosine
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [72]:
confusion_matrix = pd.DataFrame(index=images_names, columns=images_names)
confusion_matrix

,car_1.jpg,car_2.jpg,car_3.jpg,cat_1.jpg,cat_2.jpg,cat_3.jpg,truck.jpg,truck_original.jpg
car_1.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
car_2.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
car_3.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat_1.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat_2.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat_3.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
truck.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
truck_original.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
confusion_matrix.loc["car_1.jpg", "car_1.jpg"] = -1000

In [107]:
counter = 0
for image_name in images_names:
    for image_name_2 in images_names:
        print(image_name, image_name_2)
        counter += 1
        v1 = outputs[image_name].cpu().detach().numpy()
        v2 = outputs[image_name_2].cpu().detach().numpy()
        confusion_matrix.loc[image_name, image_name_2] = max(1-cosine(v1, v2),0)
print(counter)

car_1.jpg car_1.jpg
car_1.jpg car_2.jpg
car_1.jpg car_3.jpg
car_1.jpg cat_1.jpg
car_1.jpg cat_2.jpg
car_1.jpg cat_3.jpg
car_1.jpg truck.jpg
car_1.jpg truck_original.jpg
car_2.jpg car_1.jpg
car_2.jpg car_2.jpg
car_2.jpg car_3.jpg
car_2.jpg cat_1.jpg
car_2.jpg cat_2.jpg
car_2.jpg cat_3.jpg
car_2.jpg truck.jpg
car_2.jpg truck_original.jpg
car_3.jpg car_1.jpg
car_3.jpg car_2.jpg
car_3.jpg car_3.jpg
car_3.jpg cat_1.jpg
car_3.jpg cat_2.jpg
car_3.jpg cat_3.jpg
car_3.jpg truck.jpg
car_3.jpg truck_original.jpg
cat_1.jpg car_1.jpg
cat_1.jpg car_2.jpg
cat_1.jpg car_3.jpg
cat_1.jpg cat_1.jpg
cat_1.jpg cat_2.jpg
cat_1.jpg cat_3.jpg
cat_1.jpg truck.jpg
cat_1.jpg truck_original.jpg
cat_2.jpg car_1.jpg
cat_2.jpg car_2.jpg
cat_2.jpg car_3.jpg
cat_2.jpg cat_1.jpg
cat_2.jpg cat_2.jpg
cat_2.jpg cat_3.jpg
cat_2.jpg truck.jpg
cat_2.jpg truck_original.jpg
cat_3.jpg car_1.jpg
cat_3.jpg car_2.jpg
cat_3.jpg car_3.jpg
cat_3.jpg cat_1.jpg
cat_3.jpg cat_2.jpg
cat_3.jpg cat_3.jpg
cat_3.jpg truck.jpg
cat_3.jpg truck

In [108]:
confusion_matrix

,car_1.jpg,car_2.jpg,car_3.jpg,cat_1.jpg,cat_2.jpg,cat_3.jpg,truck.jpg,truck_original.jpg
car_1.jpg,1.0,0.78453,0.780421,0,0,0,0.414789,0.397632
car_2.jpg,0.78453,1.0,0.726432,0.135353,0.118959,0.096117,0.442575,0.380255
car_3.jpg,0.780421,0.726432,1.0,0,0,0,0.373716,0.38874
cat_1.jpg,0,0.135353,0,1.0,0.757223,0.692134,0.051094,0.011773
cat_2.jpg,0,0.118959,0,0.757223,1.0,0.779079,0,0
cat_3.jpg,0,0.096117,0,0.692134,0.779079,1.0,0,0
truck.jpg,0.414789,0.442575,0.373716,0.051094,0,0,1.0,0.882343
truck_original.jpg,0.397632,0.380255,0.38874,0.011773,0,0,0.882343,1.0
